# Feedback

## Load Modules

In [2]:
# make imports from pa_lib possible (parent directory of file's directory)
import sys
from pathlib import Path

file_dir = Path.cwd()
print("file_dir: -> ",file_dir)

parent_dir = file_dir.parent
print(parent_dir)

sys.path.append(str(parent_dir))

file_dir: ->  C:\Users\stc\Documents\GitHub\PredictiveAnalyticsPython\vkprog_analyse
C:\Users\stc\Documents\GitHub\PredictiveAnalyticsPython


%load_ext autoreload
%autoreload
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from pa_lib.data import (
    clean_up_categoricals
    )

from pa_lib.vis import (
    boxplot_histogram,
    )

from pa_lib.file import (
    project_dir,
    load_bin,
    load_csv,
    load_xlsx
    )
from pa_lib.util import (
    cap_words
    )                 
import os,fnmatch

https://stackabuse.com/python-list-files-in-a-directory/

# Load Data (Feedback)

In [96]:
def load_feedback(sales_folder):
    sales_subfolder = os.listdir(sales_folder)
    pattern = "EK_LIST*.xlsx"
    
    container_df = pd.DataFrame()

    for kw_folder in sales_subfolder:

        if kw_folder >= "2019_07_15": # Begin of automatisation

            all_files_list = os.listdir(sales_folder / kw_folder)

            for file in all_files_list:

                if fnmatch.fnmatch(file, pattern):
                    #print(f"{kw_folder}  ---- {file}") # Check!
                    vkber_df = (
                        load_xlsx(
                            sales_folder / kw_folder / file
                            ).iloc[:,[0,1,-3,-2,-1]])

                    vkber_df.loc[:,"folder"] = kw_folder
                    vkber_df.loc[:,"file_name"] = file

                    container_df = (
                        container_df.append(vkber_df,ignore_index=False)
                        )

    return container_df

In [95]:
sales_folder = Path('P:\Service\Kennzahlen\Verkauf\PredictiveAnalytics')
print(sales_folder)

P:\Service\Kennzahlen\Verkauf\PredictiveAnalytics


In [97]:
raw_feedback = load_feedback(sales_folder=sales_folder)

17:22:04 [INFO] Started loading xlsx file ...
17:22:04 [INFO] Reading from file \\fppwi01\daten$\Service\Kennzahlen\Verkauf\PredictiveAnalytics\2019_07_15\EK_LIST_2W_KOMPAKT_ABS.xlsx
17:22:04 [INFO] ... finished loading xlsx file in 0.04s (0.02s CPU)
17:22:04 [INFO] Started loading xlsx file ...
17:22:04 [INFO] Reading from file \\fppwi01\daten$\Service\Kennzahlen\Verkauf\PredictiveAnalytics\2019_07_15\EK_LIST_2W_KOMPAKT_ACO.xlsx
17:22:04 [INFO] ... finished loading xlsx file in 0.03s (0.02s CPU)
17:22:04 [INFO] Started loading xlsx file ...
17:22:04 [INFO] Reading from file \\fppwi01\daten$\Service\Kennzahlen\Verkauf\PredictiveAnalytics\2019_07_15\EK_LIST_2W_KOMPAKT_BAM.xlsx
17:22:04 [INFO] ... finished loading xlsx file in 0.03s (0.0s CPU)
17:22:04 [INFO] Started loading xlsx file ...
17:22:04 [INFO] Reading from file \\fppwi01\daten$\Service\Kennzahlen\Verkauf\PredictiveAnalytics\2019_07_15\EK_LIST_2W_KOMPAKT_BEG.xlsx
17:22:04 [INFO] ... finished loading xlsx file in 0.03s (0.02s CP

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


17:22:08 [INFO] Started loading xlsx file ...
17:22:08 [INFO] Reading from file \\fppwi01\daten$\Service\Kennzahlen\Verkauf\PredictiveAnalytics\2019_07_29\EK_LIST_2W_KOMPAKT_ABS.xlsx
17:22:08 [INFO] ... finished loading xlsx file in 0.03s (0.02s CPU)
17:22:08 [INFO] Started loading xlsx file ...
17:22:08 [INFO] Reading from file \\fppwi01\daten$\Service\Kennzahlen\Verkauf\PredictiveAnalytics\2019_07_29\EK_LIST_2W_KOMPAKT_ACO.xlsx
17:22:08 [INFO] ... finished loading xlsx file in 0.03s (0.02s CPU)
17:22:08 [INFO] Started loading xlsx file ...
17:22:08 [INFO] Reading from file \\fppwi01\daten$\Service\Kennzahlen\Verkauf\PredictiveAnalytics\2019_07_29\EK_LIST_2W_KOMPAKT_BAM.xlsx
17:22:08 [INFO] ... finished loading xlsx file in 0.03s (0.0s CPU)
17:22:08 [INFO] Started loading xlsx file ...
17:22:08 [INFO] Reading from file \\fppwi01\daten$\Service\Kennzahlen\Verkauf\PredictiveAnalytics\2019_07_29\EK_LIST_2W_KOMPAKT_BEG.xlsx
17:22:08 [INFO] ... finished loading xlsx file in 0.03s (0.02s CP

In [121]:
feedback_exists = ((~raw_feedback.loc[:, "Feedback - bitte auswählen"].isna()|
~raw_feedback.loc[:,"falls nicht hilfreich, bitte hier einen kurzen Kommentar angeben - entweder pro Zeile oder für die Gesamt-Liste"].isna())
                  and ~raw_feedback.loc[:,"Gepard-Nr. Endkunde"].isna())

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [119]:
raw_feedback.loc[feedback_exists,["folder","file_name","Gepard-Nr. Endkunde","Endkunde", "Chance","Feedback - bitte auswählen", "falls nicht hilfreich, bitte hier einen kurzen Kommentar angeben - entweder pro Zeile oder für die Gesamt-Liste" ]]

,folder,file_name,Gepard-Nr. Endkunde,Endkunde,Chance,Feedback - bitte auswählen,"falls nicht hilfreich, bitte hier einen kurzen Kommentar angeben - entweder pro Zeile oder für die Gesamt-Liste"
0,2019_07_15,EK_LIST_2W_KOMPAKT_ACO.xlsx,142846,Rausch AG,10.500000,hilfreich,NaN
1,2019_07_15,EK_LIST_2W_KOMPAKT_ACO.xlsx,471051,Thurbo AG,8.700000,nicht bearbeitet,NaN
2,2019_07_15,EK_LIST_2W_KOMPAKT_ACO.xlsx,573908,Stey Attraktionen GmbH,7.100000,hilfreich,NaN
3,2019_07_15,EK_LIST_2W_KOMPAKT_ACO.xlsx,566637,A.T.U Auto-Technik-Unger GmbH,7.000000,hilfreich,NaN
4,2019_07_15,EK_LIST_2W_KOMPAKT_ACO.xlsx,100373,Salzhaus GmbH,6.600000,hilfreich,NaN
5,2019_07_15,EK_LIST_2W_KOMPAKT_ACO.xlsx,325169,Zur Rose Suisse AG,5.600000,hilfreich,NaN
6,2019_07_15,EK_LIST_2W_KOMPAKT_ACO.xlsx,642379,bexio AG,4.900000,hilfreich,NaN
7,2019_07_15,EK_LIST_2W_KOMPAKT_ACO.xlsx,101714,Pfingstgemeinde Schaffhausen,4.600000,hilfreich,NaN
8,2019_07_15,EK_LIST_2W_KOMPAKT_ACO.xlsx,609448,Hutter Auto Thomi AG,4.600000,hilfreich,NaN
9,2019_07_15,EK_LIST_2W_KOMPAKT_ACO.xlsx,452472,Garage-Carrosserie Moser AG,4.200000,hilfreich,NaN


In [15]:
pattern = "EK_LIST*"
for entry in all_files_list:
    if fnmatch.fnmatch(entry, pattern):
            print (entry)

EK_LIST_2W_KOMPAKT_ACO.xlsx
EK_LIST_2W_KOMPAKT_BDJ.xlsx
EK_LIST_2W_KOMPAKT_BEB.xlsx
EK_LIST_2W_KOMPAKT_BON.xlsx
EK_LIST_2W_KOMPAKT_CBA.xlsx
EK_LIST_2W_KOMPAKT_CFU.xlsx
EK_LIST_2W_KOMPAKT_CGA.xlsx
EK_LIST_2W_KOMPAKT_CKT.xlsx
EK_LIST_2W_KOMPAKT_DCI.xlsx
EK_LIST_2W_KOMPAKT_DME.xlsx
EK_LIST_2W_KOMPAKT_DTE.xlsx
EK_LIST_2W_KOMPAKT_JAN.xlsx
EK_LIST_2W_KOMPAKT_JCO.xlsx
EK_LIST_2W_KOMPAKT_JLP.xlsx
EK_LIST_2W_KOMPAKT_LPA.xlsx
EK_LIST_2W_KOMPAKT_MBU.xlsx
EK_LIST_2W_KOMPAKT_MOE.xlsx
EK_LIST_2W_KOMPAKT_MTO.xlsx
EK_LIST_2W_KOMPAKT_MWY.xlsx
EK_LIST_2W_KOMPAKT_MZB.xlsx
EK_LIST_2W_KOMPAKT_NIE.xlsx
EK_LIST_2W_KOMPAKT_OSS.xlsx
EK_LIST_2W_KOMPAKT_PAL.xlsx
EK_LIST_2W_KOMPAKT_PIE.xlsx
EK_LIST_2W_KOMPAKT_RBE.xlsx
EK_LIST_2W_KOMPAKT_REM.xlsx
EK_LIST_2W_KOMPAKT_ROK.xlsx
EK_LIST_2W_KOMPAKT_RON.xlsx
EK_LIST_2W_KOMPAKT_RPO.xlsx
EK_LIST_2W_KOMPAKT_SAE.xlsx
EK_LIST_2W_KOMPAKT_SAO.xlsx
EK_LIST_2W_KOMPAKT_SFA.xlsx
EK_LIST_2W_KOMPAKT_SIB.xlsx
EK_LIST_2W_KOMPAKT_SOL.xlsx
EK_LIST_2W_KOMPAKT_TRA.xlsx
EK_LIST_2W_KOMPAKT_V

In [24]:
load_xlsx(deployment_folder / "EK_LIST_2W_KOMPAKT_ACO.xlsx").iloc[:,[0,1,-3,-2,-1]]

15:42:30 [INFO] Started loading xlsx file ...
15:42:30 [INFO] Reading from file \\fppwi01\daten$\Service\Kennzahlen\Verkauf\PredictiveAnalytics\2019_11_18\EK_LIST_2W_KOMPAKT_ACO.xlsx
15:42:30 [INFO] ... finished loading xlsx file in 0.03s (0.03s CPU)


,Gepard-Nr. Endkunde,Endkunde,Chance,Feedback - bitte auswählen,"falls nicht hilfreich, bitte hier einen kurzen Kommentar angeben - entweder pro Zeile oder für die Gesamt-Liste"
0,469781.0,CSL Immobilien AG,58.316522,NaN,Kunde ist in Zürich - Plakate sind abhängig vo...
1,519262.0,Hobi Moto AG,51.759579,NaN,via Ace 2 ACe
2,638475.0,Autoneum Management AG,51.302511,NaN,Autosalon Genf - Via MAT
3,515580.0,Verein Winterthurer Musikfestwochen,47.155711,NaN,"bin dran, gibt ein Sponsoring 2020"
4,139367.0,WEGA Messe AG,36.004276,NaN,Agentur wurde aufgelöst
5,605037.0,UBS (CH) Property Fund Direct Residential,34.656105,NaN,NaN
6,637276.0,Grünliberale Partei Kanton Zürich,34.426872,NaN,Wahlen 2019
7,537586.0,VIVA für Frauen,31.002352,NaN,buchen nur F4 - Winterthur -neu bei NEO
8,471431.0,Gianni's Pizzakurier,29.506658,NaN,buchen nur F4 - Winterthur -neu bei NEO
9,544744.0,inhaus Messe GmbH,29.473159,NaN,Agentur wurde aufgelöst
